#Octave simple PBPK model for Styrene dosing by R C
#Ported to Jupyter Notebook on 9/3/2018 by Mo M

%Simple PBPK model coded in Octave
%Rory Conolly
%January 16, 2018

%This is a relatively simple PBPK model developed for teaching purposes. 
%The model consists of liver, richly perfused, slowly perfused, 
%fat and blood compartments, and a non-physiological dosing compartment for
%oral dosing. The physiological parameters are for a mouse, and the 
%chemical-specific parameters for styrene.   

%INITIAL
global c; %Anything that starts with "c." is available to all parts of the program
clc %clears the screen
disp('Starting...')

%Mouse physiology (from Leung et al., Toxicol. Appl. Pharmacol. 103, 411-419, 1990)
BW = 0.03; 		%body weight (kg) (30 g)
%Tissues as percent of BW (assuming unit density, i.e., 1 gm equivalent to 1 ml)
VLC = 5; 		%liver
VRC = 4; 		%richly perfused - the other viscera
VSC = 76.1; 	%slowly perfused - muscle, skin
VFC = 5.9; 		%fat
VBC = 5;  		%blood
%note that individual tissue weights sum to only 96 percent of BW. The remaining 4
%percent is assumed to consist of structural components with minimal blood flow 
%(see Ramsey and Andersen, Toxicol. Appl. Pharmacol. 73, 159-175, 1984)
c.VL = VLC*BW/100; 	%liver (kg)
c.VR = VRC*BW/100; 	%richly perfused (kg)
c.VS = VSC*BW/100; 	%slowly perfused (kg)
c.VF = VFC*BW/100; 	%fat (kg)
c.VB = VBC*BW/100; 	%blood (kg)
%Blood flow 
c.QC = 1.045; 	%cardiac output (L/hr)
%Tissue blood flows as percent of QC
QLC = 25; 		%liver
QRC = 51; 		%richly perfused
QSC = 17; 		%slowly perfussed
QFC = 7; 		%fat
c.QL = QLC*c.QC/100; 	%liver (L/hr)
c.QR = QRC*c.QC/100; 	%richaly perfused (L/hr)
c.QS = QSC*c.QC/100; 	%slowly perfused (L/hr)
c.QF = QFC*c.QC/100; 	%fat (L/hr)

%Chemical-specific parameters for styene
%MW = 104.15; Molecular weight of styrene (g/mol)
% Styrene is metabolized in the liver by Michealis-Menten kinetics
c.Vmax = 1.0092; %(mg styrene metabolized in liver/hr)
%This value for the Vmax in the mouse is calculated from the value for the rat
%assuming that the value scales as BW^0.75, which roughly means scaling with 
%body surface area. Here is the calculation:
%Vmax_rat = 4.95 mg/hr
%VmaxC = Vmax_rat/(BWrat^0.75), BWrat = 0.25 kg
%Vmax_mouse = VmaxC * BWmouse^0.75
c.Km = 0.4; 	% Assume same as rat (mg/L) 
% Partition coefficients (ratio of concentrations at steady state) 
c.pl = 3.46;	%Liver/blood partition coefficient
c.pr = 3.46;   	%Richly perfused/blood partition coefficient
c.ps = 1.16;	%Slowly perfused/blood partition coefficient	
c.pf = 50.;    	%Fat/blood partition coefficient

c.kabs_dose = 0.1; %absorption rate constant (1/hr)

%Design of experiment
tstop = 24;			%24 hr
tspan = linspace(0, tstop, tstop * 10); %Interval over which the ODEs are solved
dose_styrene = 10;	% mg dosed orally

%Prepare inputs to LSODE
A_liver = 0; 	%initial amount (mg)
A_rich = 0;		%initial amount (mg)
A_slow = 0;		%initial amount (mg)
A_fat = 0;		%initial amount (mg)
A_blood = 0;	%initial amount (mg)
A_met = 0;		%initial amount styrene metabolized by liver (mg)
y0 = [dose_styrene A_liver A_rich A_slow A_fat A_blood A_met];

%LSODE options
lsode_options("absolute tolerance",1e-15);
lsode_options("relative tolerance",1.e-4);
lsode_options("integration method","stiff");
lsode_options("maximum order", 5);
lsode_options("maximum step size", 0.1);
lsode_options("minimum step size", 1.e-15);

%END of Initial

%Dynamic
simdata = lsode('PBPK_ode_v1',y0,tspan);
			%"Two_compartment_ode" specifies the model structure

%END of Dynamic

%Postprocessing

amount_dose = simdata(:,1);%amount dosing compartment (mg)
conc_liver = simdata(:,2)/c.VL; %conc. liver (mg/L)
conc_rapid = simdata(:,3)/c.VR; %conc. liver (mg/L)
conc_slow = simdata(:,4)/c.VS;   %conc. slowly perf. (mg/L)
conc_fat = simdata(:,5)/c.VF; %conc. fat (mg/L)
conc_blood = simdata(:,6)/c.VB; %conc. blood (mg/L)
amount_met = simdata(:,7); %amount metabolized (mg)

subplot(2,3,1), plot(tspan, amount_dose)
xlabel('Hours')
ylabel('Amount in dosing compartment (mg)')
subplot(2,3,2), plot(tspan, amount_met)
xlabel('Hours')
ylabel('Amoount metabolized (mg)')
subplot(2,3,3), plot(tspan, conc_liver)
xlabel('Hours')
ylabel('Conc liver (mg/L)')
subplot(2,3,4), plot(tspan, conc_slow)
xlabel('Hours')
ylabel('Conc slow (mg/L)')
subplot(2,3,5), plot(tspan, conc_fat)
xlabel('Hours')
ylabel('Conc fat (mg/L)')
subplot(2,3,6), plot(tspan, conc_blood)
xlabel('Hours')
ylabel('Conc blood (mg/L)')

disp('Done')

%END of program

%Simple PBPK model coded in Octave
%Rory Conolly
%January 16, 2018

%This is a relatively simple PBPK model developed for teaching purposes. 
%The model consists of liver, richly perfused, slowly perfused, 
%fat and blood compartments, and a non-physiological dosing compartment for
%oral dosing. The physiological parameters are for a mouse, and the 
%chemical-specific parameters for styrene.   

%INITIAL
global c; %Anything that starts with "c." is available to all parts of the program
clc %clears the screen
disp('Starting...')

%Mouse physiology (from Leung et al., Toxicol. Appl. Pharmacol. 103, 411-419, 1990)
BW = 0.03; 		%body weight (kg) (30 g)
%Tissues as percent of BW (assuming unit density, i.e., 1 gm equivalent to 1 ml)
VLC = 5; 		%liver
VRC = 4; 		%richly perfused - the other viscera
VSC = 76.1; 	%slowly perfused - muscle, skin
VFC = 5.9; 		%fat
VBC = 5;  		%blood
%note that individual tissue weights sum to only 96 percent of BW. The remaining 4
%percent is assumed to consist of structural components with minimal blood flow 
%(see Ramsey and Andersen, Toxicol. Appl. Pharmacol. 73, 159-175, 1984)
c.VL = VLC*BW/100; 	%liver (kg)
c.VR = VRC*BW/100; 	%richly perfused (kg)
c.VS = VSC*BW/100; 	%slowly perfused (kg)
c.VF = VFC*BW/100; 	%fat (kg)
c.VB = VBC*BW/100; 	%blood (kg)
%Blood flow 
c.QC = 1.045; 	%cardiac output (L/hr)
%Tissue blood flows as percent of QC
QLC = 25; 		%liver
QRC = 51; 		%richly perfused
QSC = 17; 		%slowly perfussed
QFC = 7; 		%fat
c.QL = QLC*c.QC/100; 	%liver (L/hr)
c.QR = QRC*c.QC/100; 	%richaly perfused (L/hr)
c.QS = QSC*c.QC/100; 	%slowly perfused (L/hr)
c.QF = QFC*c.QC/100; 	%fat (L/hr)

%Chemical-specific parameters for styene
%MW = 104.15; Molecular weight of styrene (g/mol)
% Styrene is metabolized in the liver by Michealis-Menten kinetics
c.Vmax = 1.0092; %(mg styrene metabolized in liver/hr)
%This value for the Vmax in the mouse is calculated from the value for the rat
%assuming that the value scales as BW^0.75, which roughly means scaling with 
%body surface area. Here is the calculation:
%Vmax_rat = 4.95 mg/hr
%VmaxC = Vmax_rat/(BWrat^0.75), BWrat = 0.25 kg
%Vmax_mouse = VmaxC * BWmouse^0.75
c.Km = 0.4; 	% Assume same as rat (mg/L) 
% Partition coefficients (ratio of concentrations at steady state) 
c.pl = 3.46;	%Liver/blood partition coefficient
c.pr = 3.46;   	%Richly perfused/blood partition coefficient
c.ps = 1.16;	%Slowly perfused/blood partition coefficient	
c.pf = 50.;    	%Fat/blood partition coefficient

c.kabs_dose = 0.1; %absorption rate constant (1/hr)

%Design of experiment
tstop = 24;			%24 hr
tspan = linspace(0, tstop, tstop * 10); %Interval over which the ODEs are solved
dose_styrene = 10;	% mg dosed orally

%Prepare inputs to LSODE
A_liver = 0; 	%initial amount (mg)
A_rich = 0;		%initial amount (mg)
A_slow = 0;		%initial amount (mg)
A_fat = 0;		%initial amount (mg)
A_blood = 0;	%initial amount (mg)
A_met = 0;		%initial amount styrene metabolized by liver (mg)
y0 = [dose_styrene A_liver A_rich A_slow A_fat A_blood A_met];

%LSODE options
lsode_options("absolute tolerance",1e-15);
lsode_options("relative tolerance",1.e-4);
lsode_options("integration method","stiff");
lsode_options("maximum order", 5);
lsode_options("maximum step size", 0.1);
lsode_options("minimum step size", 1.e-15);

%END of Initial

%Dynamic
simdata = lsode('PBPK_ode_v1',y0,tspan);
			%"Two_compartment_ode" specifies the model structure

%END of Dynamic

%Postprocessing

amount_dose = simdata(:,1);%amount dosing compartment (mg)
conc_liver = simdata(:,2)/c.VL; %conc. liver (mg/L)
conc_rapid = simdata(:,3)/c.VR; %conc. liver (mg/L)
conc_slow = simdata(:,4)/c.VS;   %conc. slowly perf. (mg/L)
conc_fat = simdata(:,5)/c.VF; %conc. fat (mg/L)
conc_blood = simdata(:,6)/c.VB; %conc. blood (mg/L)
amount_met = simdata(:,7); %amount metabolized (mg)

subplot(2,3,1), plot(tspan, amount_dose)
xlabel('Hours')
ylabel('Amount in dosing compartment (mg)')
subplot(2,3,2), plot(tspan, amount_met)
xlabel('Hours')
ylabel('Amoount metabolized (mg)')
subplot(2,3,3), plot(tspan, conc_liver)
xlabel('Hours')
ylabel('Conc liver (mg/L)')
subplot(2,3,4), plot(tspan, conc_slow)
xlabel('Hours')
ylabel('Conc slow (mg/L)')
subplot(2,3,5), plot(tspan, conc_fat)
xlabel('Hours')
ylabel('Conc fat (mg/L)')
subplot(2,3,6), plot(tspan, conc_blood)
xlabel('Hours')
ylabel('Conc blood (mg/L)')

disp('Done')

%END of program